This function computes the (linear) interpolant for evaluating the differential equation:

\begin{equation}
\phi' = - \dfrac{\cos^2(\phi)[S_x^{11}(\mathbf{x})\cos(\phi)+S_y^{11}(\mathbf{x})\sin(\phi)]+[S_x^{12}(\mathbf{x})\cos(\phi)+S_y^{11}(\mathbf{x})\sin(\phi)]\sin(2\phi)+\sin^2(\phi)[S_x^{22}(\mathbf{x})\cos(\phi)+S_y^{22}(\mathbf{x})\sin(\phi)}{\sin(2\phi)[S^{22}(\mathbf{x})-^{11}(\mathbf{x})]+2\cos(2\phi)S^{12}(\mathbf{x})}
\label{eq: phi_prime}
\end{equation}

at every point $ (\mathbf{x}, \phi) $. To do so, we first need to evaluate the right-hand-side of eq. \ref{eq: phi_prime} over a rectangular meshgrid $ X, Y, \phi $ (see *__phi_prime*) before computing the interpolant of $ \dot{\phi} $ (see *_phi_prime*).

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import cos, sin, pi

# Import numba (used for high performance scientific computing)
from numba import jit, prange

In [1]:
@jit(parallel=True)
def __phi_prime(X, Y, phi, S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    
    phi_prime = np.zeros((Y.shape[0], X.shape[1], phi.shape[0]))
    phi_prime_denom = phi_prime.copy()
    
    for i in prange(Y.shape[0]):
        
        for j in prange(X.shape[1]):
            
            S11_ = S11[i, j]
            S12_ = S12[i, j]
            S22_ = S22[i, j]
            
            S11_x_ = S11_x[i, j]
            S11_y_ = S11_y[i, j]
            
            S12_x_ = S12_x[i, j]
            S12_y_ = S12_y[i, j]
            
            S22_x_ = S22_x[i, j]
            S22_y_ = S22_y[i, j]

            for p in prange(phi.shape[0]):
                
                phi_ = phi[p]
    
                phi_prime_num = (cos(phi_)*S11_x_+S11_y_*sin(phi_))*cos(phi_)**2
                phi_prime_num += (cos(phi_)*S12_x_+S12_y_*sin(phi_))*sin(2*phi_)
                phi_prime_num += (cos(phi_)*S22_x_+S22_y_*sin(phi_))*sin(phi_)**2
    
                phi_prime_denom[i, j, p] = sin(2*phi_)*(S22_-S11_)+2*cos(2*phi_)*S12_
            
                phi_prime[i, j, p] = -phi_prime_num/phi_prime_denom[i, j, p]

    return phi_prime, phi_prime_denom

In [2]:
def _phi_prime(X, Y, S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    
    phi = np.linspace(0, 2*pi, 360)
    
    # Import interpolator from scipy
    from scipy.interpolate import RegularGridInterpolator
    
    phi_prime, DOE = __phi_prime(X, Y, phi, S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y)
    
    return RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime), RegularGridInterpolator((Y[:,0], X[0,:], phi), DOE)